<h1> Earthquake Cycle Simulations

In [ ]:
# Plot friction parameters
using Gadfly

fricPlot = plot(layer(x = (cca-ccb).*1e3 + 10, y = FltX/1e3,Theme(default_color=colorant"red"), Geom.point),
                layer(x = Seff/1e6, y = FltX/1e3, Theme(default_color=colorant"deepskyblue"), Geom.point),
                layer(x = tauo/1e6, y = FltX/1e3, Theme(default_color=colorant"green"), Geom.point),
                Guide.xlabel("Scaled (a-b)/ Stress value"),
                Guide.ylabel("Depth (m)"),
                Guide.title("Rate and state friction/Stress"),
                Coord.Cartesian(ymin=-24))

In [ ]:
@time include("src/setup.jl")
@time include("src/timesolver.jl")

In [1]:
include("src/setup.jl")

Total number of nodes on fault = 121
Average node spacing = 400.0
dt = 0.04486312014474891
Setup Complete


In [2]:
include("src/PCG.jl")
include("src/dtevol.jl")
include("src/NRsearch.jl")
include("src/otherFunctions.jl")

vPre = v[:]
        dPre = d[:]

        Vf0 = 2*v[iFlt] + Vpl
        Vf = Vf0[:];

In [6]:
a

9801×1 Array{Float64,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [9]:
F[:] = 0
            F[iFlt] = dPre[iFlt] + v[iFlt]*dt

            # Assign previous displacement field as initial guess
            dnew = d[FltNI]

            # Solve d = K^-1F by PCG
            dnew = PCG(coefint1, coefint2, diagKnew, dnew, F, iFlt, FltNI,
                          H, Ht, iglob, Nel, nglob, W);

n = 4000


LoadError: [91mPCG did not converge[39m

In [8]:
a

9801×1 Array{Float64,2}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮  
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [ ]:
 # update displacement on the medium
            d[FltNI] = dnew

            # make d = F on the fault
            d[iFlt] = F[iFlt]

            # Compute on-fault stress
            a[:] = 0
            for eo = 1:Nel
                ig = iglob[:,:,eo]
                locall = d[ig]

                # Gradients wrt local variables
                d_xi = Ht*locall
                d_eta = locall*H

                # Element contribution
                wloc = W[:,:,eo]
                d_xi = wloc.*d_xi
                d_xi = H*d_xi
                d_eta = wloc.*d_eta
                d_eta = d_eta*Ht
                locall = coefint1*d_xi + coefint2*d_eta

                # Assemble into global vector
                a[ig] = a[ig] + locall
            end


In [ ]:
a[FltIglobBC] = 0
            tau1 = -a[iFlt]./FltB

In [ ]:
for jF = 1:FaultNglob-NFBC 

                j = jF - 1 + NFBC
                psi1[j] = IDS(psi[j], dt, Vo[j], xLf[j], Vf[j], 1e-6)

                tauAB[j] = tau1[j] + tauo[j]
                fa = tauAB[j]/(Seff[j]*cca[j])
                help = -(fo[j] + ccb[j]*psi1[j])/cca[j]
                help1 = exp(help + fa)
                help2 = exp(help - fa)
                Vf1[j] = Vo[j]*(help1 - help2) 
            end

In [ ]:
Vf1

In [ ]:
 Vf1[iFBC] = Vpl
            Vf = (Vf0 + Vf1)/2
            v[iFlt] = 0.5*(Vf - Vpl);

<h3> PCG debug

In [ ]:
 global a 
    a[:] =0 # zeros(nglob)
    dd = zeros(nglob)
    p = zeros(nglob)
    
    a = element_computation(Nel, iglob, F, H, Ht, coefint1, coefint2, W)
    Fnew = -a[FltNI]

    dd[FltNI] = dnew
    dd[iFlt] = 0

    a[:] = 0
    
    a = element_computation(Nel, iglob, dd, H, Ht, coefint1, coefint2, W)
    anew = a[FltNI]

    # Initial residue
    rnew = Fnew - anew
    znew = rnew./diagKnew
    pnew = znew
    p[:] = 0
    p[FltNI] = pnew

    for n = 1:4000
        anew[:] = 0
        a[:] = 0
        
        a = element_computation(Nel, iglob, p, H, Ht, coefint1, coefint2, W)

        anew = a[FltNI]
        alpha_ = znew'*rnew/(pnew'*anew)
        dnew = dnew + alpha_*pnew
        rold = rnew
        zold = znew
        rnew = rold - alpha_*anew
        znew = rnew./diagKnew
        beta_ = znew'*rnew/(zold'*rold)
        pnew = znew + beta_*pnew
        p[:] = 0
        p[FltNI] = pnew

        if norm(rnew)/norm(Fnew) < 1e-5
            break;
        end

        if n == 4000 || norm(rnew)/norm(Fnew) > 1e10
            println("n = ", n)
            error("PCG did not converge")
            return
        end
    end

In [ ]:
znew